**About :** Trains XGBoost models.

**TODO**:
- better neg sampling technique ??

In [1]:
cd ../src

/workspace/kaggle_otto_rs/src


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
import os
import gc
import cudf
import json
import glob
import numba
import xgboost
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from numerize.numerize import numerize
from pandarallel import pandarallel

warnings.simplefilter(action="ignore", category=FutureWarning)
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
pandarallel.initialize(nb_workers=32, progress_bar=False)

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
from params import *

from utils.metrics import get_coverage, evaluate
from utils.plot import plot_importances
from utils.load import *
from utils.logger import *

from inference.xgb import xgb_inference

### Inf

In [6]:
EXP_FOLDER = "../logs/2023-01-24/27/"  #  "../logs/2023-01-26/3/"  # "../logs/2023-01-26/2/"
VERSION = "cv7-tv5.11"  # 0.6692 /  0.6716 /  0.6684 / 0.6693 -> 0.6696
# VERSION = "cv8-tv5.11"  # 0.6695 /  0.6710 /  0.6686 / 0.6694 -> 0.6696
# VERSION = "cv9-tv5.11"  # 0.6690 /  0.6717 /  0.6683 / 0.6691 -> 0.6695

REGEX = f"../output/features/fts_val_{VERSION}/*"
TEST_REGEX = None  # f"../output/features/fts_test_{VERSION}/*"

In [7]:
# xgb_inference(REGEX, TEST_REGEX, EXP_FOLDER, debug=False, save=True)  # CV 0.6696  //  0.6692 - 0.6716 - 0.6684 - 0.6693

In [8]:
# ft_imp = pd.read_csv("../logs/2023-01-24/22/ft_imp.csv")
# list(ft_imp.sort_values('importance')["index"])[:200]
# plot_importances(ft_imp.set_index("index"))

In [9]:
REGEX = "../output/val_parquet/*"

sessions = load_sessions(REGEX)
sessions_g = sessions[["session", "aid"]].groupby('session').agg("count").reset_index()
sessions_len_1 = sessions_g[sessions_g['aid'] == 1]

sessions_len_more = sessions_g[sessions_g['aid'] > 1]
# sessions_g = sessions_g.merge(sessions_g2, on="session", suffixes=('', '_'), how="left")
# sessions_len_1 = sessions_g.dropna(0)

### Load

#### Train data
- neg sampling could use candidates from lower versions

In [10]:
EXP_FOLDERS = [  # ORDERS - 0.6680
    ["../logs/2023-01-20/1/", "../logs/2023-01-20/9/", "../logs/2023-01-20/13/", "../logs/2023-01-20/12/"],
]

In [11]:
EXP_FOLDERS = [  # Carts - 0.4404
    ["../logs/2023-01-20/17/", "../logs/2023-01-20/29/", "../logs/2023-01-20/28/", "../logs/2023-01-20/27/"],
]

In [12]:
EXP_FOLDERS = [  # Clicks - 0.5593
    ["../logs/2023-01-20/26/", "../logs/2023-01-20/23/", "../logs/2023-01-20/30/", "../logs/2023-01-20/32/"],
]

In [13]:
EXP_FOLDERS = [
    "../logs/2023-01-24/29/",  # CARTS - Pos ratio 0.2 - Extra_prop 0                CV 0.4420
    ["../logs/2023-01-25/0/", "../logs/2023-01-25/4/",  "../logs/2023-01-25/6/", "../logs/2023-01-25/7/"],  # CARTS Pos ratio 0.5 - Extra_prop 1.0    CV 0.4424
    ["../logs/2023-01-26/3/", "../logs/2023-01-26/2/"],                                                     # CARTS Pos ratio 0.5 - Extra_prop 2.0    CV 0.4426
]

WEIGHTS = [1, 1, 2]  # Carts 0.4428

In [48]:
EXP_FOLDERS = [ # Orders 0.6698
    "../logs/2023-01-24/28/",  # ORDER - Pos ratio 0.2 - Extra_prop 0.5              CV 0.6694
    "../logs/2023-01-24/27/",  # ORDER - Pos ratio 0.5 - Extra_prop 1                CV 0.6696
    "../logs/2023-01-24/17/",  # ORDER - Pos ratio 0.2 - Extra_prop 0.5  rm low imp  CV 0.6694
    "../logs/2023-01-25/11/",  # Pos ratio 0.5 - Extra_prop 2.0                       CV 0.6697
]

WEIGHTS = [0.25, 2, 0.25, 2]

In [45]:
EXP_FOLDERS = [
#     "../logs/2023-01-24/29/",  # CARTS - Pos ratio 0.2 - Extra_prop 0                CV 0.4420
    "../logs/2023-01-24/28/",  # ORDER - Pos ratio 0.2 - Extra_prop 0.5              CV 0.6694
    "../logs/2023-01-24/27/",  # ORDER - Pos ratio 0.5 - Extra_prop 1                CV 0.6696
#     "../logs/2023-01-24/22/",  # ORDER - Pos ratio 0.2 - Extra_prop 0                CV 0.6691
    "../logs/2023-01-24/17/",  # ORDER - Pos ratio 0.2 - Extra_prop 0.5  rm low imp  CV 0.6694
#     ["../logs/2023-01-25/0/", "../logs/2023-01-25/4/",  "../logs/2023-01-25/6/", "../logs/2023-01-25/7/"],  # CARTS Pos ratio 0.5 - Extra_prop 1.0    CV 0.4424
#     ["../logs/2023-01-26/3/", "../logs/2023-01-26/2/"],                                                     # CARTS Pos ratio 0.5 - Extra_prop 2.0    CV 0.4426
    
    "../logs/2023-01-25/11/",  # Pos ratio 0.5 - Extra_prop 2.0                       CV 0.6697
]

WEIGHTS = [1] * len(EXP_FOLDERS)

In [49]:
df_val_ = None
# dfs_val_list = []
for exp_folders, w in zip(EXP_FOLDERS, WEIGHTS):
    
    if not isinstance(exp_folders, list):
        exp_folders = [exp_folders]

    dfs_val = []
    for exp_folder in exp_folders:
        config = Config(json.load(open(exp_folder + "config.json", "r")))
        print(f'\t === Exp {exp_folder}\t Target {config.target} ===\n')
        TARGET = config.target
        try:
            print(f"{config.version} - Pos ratio {config.pos_ratio} - Extra_prop {config.extra_prop if config.use_extra else 0}\n")
        except:
            print(f"{config.version} - Pos ratio {config.pos_ratio} - Extra_prop 0\n")

        for fold in range(config.k):
#             if fold == 3:
#                 continue
            try:
                df_val = cudf.read_parquet(exp_folder + f"df_val_{fold}.parquet")
#                 df_val = df_val.merge(sessions_len_1, on="session", how="left").dropna(0)
#                 df_val = df_val.merge(sessions_len_more, on="session", how="left").dropna(0)

#                 df_val['pred'] = df_val.groupby('session')['pred'].rank()
#                 df_val['pred'] = (df_val['pred'] - df_val['pred'].min()) / (df_val['pred'].max() - df_val['pred'].min())
                df_val['pred'] *= w

                dfs_val.append(df_val)
#                 print(f'-> Retrieved fold {fold}', end="")
#                 evaluate(df_val, TARGET)
            except FileNotFoundError:
                pass
            
#             break

    dfs_val = cudf.concat(dfs_val, ignore_index=True)
#     dfs_val_list.append(dfs_val.sort_values(['session', 'candidates'], ignore_index=True))

#     print('\n ===> CV :')
#     evaluate(dfs_val, TARGET)    

    if df_val_ is None: 
        df_val_ = dfs_val.copy()
    else:
        df_val_ = df_val_.set_index(['session', 'candidates']).add(
            dfs_val.set_index(['session', 'candidates']), fill_value=0
        ).reset_index()

    del dfs_val
    numba.cuda.current_context().deallocations.clear()
    gc.collect()

	 === Exp ../logs/2023-01-24/28/	 Target gt_orders ===

cv7-tv5.11 - Pos ratio 0.2 - Extra_prop 0.5

	 === Exp ../logs/2023-01-24/27/	 Target gt_orders ===

cv7-tv5.11 - Pos ratio 0.5 - Extra_prop 2

	 === Exp ../logs/2023-01-24/17/	 Target gt_orders ===

cv7-tv5.11 - Pos ratio 0.2 - Extra_prop 0.5

	 === Exp ../logs/2023-01-25/11/	 Target gt_orders ===

cv7-tv5.11 - Pos ratio 0.5 - Extra_prop 2.0



In [50]:
evaluate(df_val_, TARGET)


-> gt_orders  -  Recall : 0.6698



0.6698090920699783

### Stack

In [51]:
from cuml import LogisticRegression

def kfold(df, config):
    seed_everything(config.seed)

    folds = cudf.read_csv(config.folds_file)
    if "fold" not in df.columns:
        df = df.merge(folds, how="left", on="session")

    preds = []
    for fold in range(config.k):
        df_train = df[df['fold'] != fold].reset_index(drop=True)
        df_val = df[df['fold'] == fold].reset_index(drop=True)

        model = LogisticRegression(C=0.1)
        cols = ['pred_0', 'rank_0', 'pred_1', 'rank_1']

        model.fit(df_train[cols], df_train[TARGET])
        df_val['pred'] = model.predict_proba(df_val[cols])[1]

#         evaluate(df_val, TARGET)
        preds.append(df_val)
#         return
        
    preds = cudf.concat(preds, ignore_index=True)
    evaluate(preds, TARGET)

In [52]:
# dfs_vals = dfs_val_list[0].merge(dfs_val_list[1], on=['session', 'candidates'], how="left", suffixes=("_0", "_1"))
# dfs_vals = dfs_vals.dropna(0)

# gt = cudf.read_parquet("../output/candidates/candidates_cv3-tv5_val.parquet")
# # gt = cudf.read_parquet("../output/candidates/candidates_cv7-tv5_val.parquet")
# dfs_vals = dfs_vals.merge(gt, how="left")

In [53]:
# dfs_vals['pred'] = dfs_vals['pred_1']
# evaluate(dfs_vals, TARGET)

In [54]:
# kfold(dfs_vals, config)

### Subs

In [55]:
from params import WEIGHTS
import numpy as np

In [56]:
cv = np.average([0.5593, 0.4428, 0.6698], weights=WEIGHTS)  # LB 0.601+
print(f"-> CV : {cv:.4f}")

-> CV : 0.5907


In [57]:
cv = np.average([0.5593, 0.4424, 0.6694], weights=WEIGHTS)  # LB 0.601+
print(f"-> CV : {cv:.4f}")

-> CV : 0.5903


In [58]:
cv = np.average([0.5593, 0.4420, 0.6691], weights=WEIGHTS)  # LB 0.601
print(f"-> CV : {cv:.4f}")

-> CV : 0.5900


In [ ]:
cv = np.average([0.5593, 0.4404, 0.6680], weights=WEIGHTS)  # LB 0.600
print(f"-> CV : {cv:.4f}")

In [ ]:
cv = np.average([0.554, 0.4404, 0.6680], weights=WEIGHTS)  # LB 0.599  - High
print(f"-> CV : {cv:.4f}")  

In [ ]:
cv = np.average([0.554, 0.4392, 0.6673], weights=WEIGHTS)  # LB 0.599  - Low 
print(f"-> CV : {cv:.4f}")

In [ ]:
cv = np.average([0.554, 0.4382, 0.6668], weights=WEIGHTS)  # LB 0.598  - Mid
print(f"-> CV : {cv:.4f}")

### Test

In [ ]:
df_test = None
for exp_folders in EXP_FOLDERS:
    dfs_test = []

    if not isinstance(exp_folders, list):
        exp_folders = [exp_folders]
    
    for exp_folder in exp_folders:
        config = Config(json.load(open(exp_folder + "config.json", "r")))
        print(f'\t === Exp {exp_folder}\t Target {config.target} ===\n')
        TARGET = config.target

        for fold in range(config.k):
            try:
                dfs_test.append(cudf.read_parquet(exp_folder + f"df_test_{fold}.parquet"))
                print(f'-> Retrieved fold {fold}\n')
            except FileNotFoundError:
                pass

    dfs_test = cudf.concat(dfs_test, ignore_index=True).groupby(['session', 'candidates']).mean().reset_index()

    print(f'- Retrieved {len(dfs_test)} test candidates.\n')

    if df_test is None:
        df_test = dfs_test
    else:
        df_test = df_test.set_index(['session', 'candidates']).add(
            dfs_test.set_index(['session', 'candidates']), fill_value=0
        ).reset_index()

    del dfs_test
    numba.cuda.current_context().deallocations.clear()
    gc.collect()

In [ ]:
preds = df_test[['session', 'candidates', 'pred']].copy()

preds = preds.sort_values(['session', 'pred'], ascending=[True, False])
preds = preds[['session', 'candidates', 'pred']].groupby('session').agg(list).reset_index()

preds = preds.to_pandas()
preds['candidates'] = preds['candidates'].apply(lambda x: x[:20])

In [ ]:
# Fill less than 20 candidates. This should be useless in the future
dfs = load_sessions(f"../output/test_parquet/*")

if config.target == "gt_carts":
    top = dfs.loc[dfs["type"] == 1, "aid"].value_counts().index.values[:20].tolist()
elif config.target == "gt_orders":
    top = dfs.loc[dfs["type"] == 2, "aid"].value_counts().index.values[:20].tolist()
else:
    top = dfs.loc[dfs["type"] == 0, "aid"].value_counts().index.values[:20].tolist()

preds['candidates'] = preds['candidates'].apply(lambda x: list(x) + top[: 20 - len(x)])

del dfs
numba.cuda.current_context().deallocations.clear()
gc.collect()

### Save

In [ ]:
FT_VERSION = "cv7-tv5.11"
MODEL_VERSION = "2"

In [ ]:
log_folder_2 = LOG_PATH + f"{FT_VERSION}.{MODEL_VERSION}/"

os.makedirs(log_folder_2, exist_ok=True)
save_config(config, log_folder_2 + 'config')

In [ ]:
sub = preds[['session', 'candidates']].copy()
assert len(sub) == 1671803

sub['candidates'] = sub['candidates'].parallel_apply(lambda x: " ".join(map(str, x)))
sub['session'] =  sub['session'].astype(str) + "_" + TARGET[3:]
sub.columns = ["session_type", "labels"]

sub.to_csv(log_folder_2 + f'sub_{TARGET}.csv', index=False)
print(f"-> Saved sub to {log_folder_2 + f'sub_{TARGET}.csv'}\n")

display(sub.head())

In [ ]:
if all([os.path.exists(log_folder_2 + f'sub_gt_{c}.csv') for c in CLASSES]):
    sub_final = cudf.concat([
        cudf.read_csv(log_folder_2 + f'sub_gt_{c}.csv') for c in CLASSES
    ], ignore_index=True)

    assert len(sub_final) == 5015409
    sub_final.to_csv(log_folder_2 + f"submission.csv", index=False)

    print(f"\n-> Saved final sub to {log_folder_2 + f'submission.csv'}\n")

    display(sub_final.sample(5))

In [ ]:
# kaggle competitions submit -c otto-recommender-system -f submission.csv -m "Message"

Done